In [ ]:
# Immport
from xml.dom import minidom
import bs4 as bs
import os
from pathlib import Path
import glob
from tqdm import tqdm
import random
import shutil

In [ ]:
def convertPascal2YOLOv8(filePath):

    class_mapping = {
        "D00": 0,
        "D10": 1,
        "D20": 2,
        "D40": 3,
        "D43": 4,
        "D44": 5,
        "D50": 6,
        "Repair": 7
    }

    # reading content
    file = open(filePath, "r")
    contents = file.read()


    soup = bs.BeautifulSoup(contents, 'xml')
    image_size = soup.find_all("size")[0]
    image_width = int(image_size.find_all("width")[0].get_text())
    image_height = int(image_size.find_all("height")[0].get_text())
    # print("w,h :", image_width, image_height)

    # Process Bounding Box
    objects = soup.find_all("object")

    # Placeholder
    bounding_box_list = []
    class_list = []

    for object in objects:

        # Object Class
        _class = object.find_all("name")[0].get_text()


        _class = class_mapping.get(_class, 10)
        class_list.append(_class)


        _xmin = float(object.find_all("xmin")[0].get_text())
        _ymin = float(object.find_all("ymin")[0].get_text())
        _xmax = float(object.find_all("xmax")[0].get_text())
        _ymax = float(object.find_all("ymax")[0].get_text())



        w = (_xmax - _xmin)
        h = (_ymax - _ymin)
        cx = _xmin + (w/2)
        cy = _ymin + (h/2)

        # Normalize
        w = round((w / image_width), 4)
        h = round((h / image_height), 4)
        cx = round((cx / image_width), 4)
        cy = round((cy / image_height), 4)

        _bbox = [cx, cy, w, h]

        # print(_class, cx, cy, w, h)

        bounding_box_list.append(_bbox)

    # Get the filename
    outputFilename = os.path.split(filePath)[1]
    outputFilename = outputFilename.replace(".xml", ".txt")

    # Output Path
    outputDir = Path(filePath).parents[2]
    outputDir = outputDir / "labels"

    # Check if the directory exists
    if not os.path.exists(outputDir):
        os.makedirs(outputDir)

    # Final output path
    outputPath = outputDir / outputFilename
    # print(outputPath)

    # Write to .txt file
    with open(outputPath, 'w') as f:
        for i in range(len(class_list)):

            anno = str(class_list[i]) + " " +  str(bounding_box_list[i][0]) + " " +  str(bounding_box_list[i][1]) + " " +  str(bounding_box_list[i][2]) + " " +  str(bounding_box_list[i][3]) + "\n"
            f.write(anno)


In [ ]:
!unzip -o 'RDD2022.zip' -d /content/RDD2022

In [ ]:
import zipfile
import os
import glob


ROOTDIR = '/content/RDD2022/RDD2022_all_countries/'


zip_files = glob.glob(os.path.join(ROOTDIR, '*.zip'))


for zip_file in zip_files:
    file_name = os.path.basename(zip_file).replace('.zip', '')
    extract_dir = os.path.join(ROOTDIR, file_name)
    print(f"Extracting {zip_file} to {extract_dir}")


    os.makedirs(extract_dir, exist_ok=True)


    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)


!ls -lh /content/RDD2022/RDD2022_all_countries

Extracting /content/RDD2022/RDD2022_all_countries/Japan.zip to /content/RDD2022/RDD2022_all_countries/Japan
Extracting /content/RDD2022/RDD2022_all_countries/China_MotorBike.zip to /content/RDD2022/RDD2022_all_countries/China_MotorBike
Extracting /content/RDD2022/RDD2022_all_countries/China_Drone.zip to /content/RDD2022/RDD2022_all_countries/China_Drone
Extracting /content/RDD2022/RDD2022_all_countries/United_States.zip to /content/RDD2022/RDD2022_all_countries/United_States
Extracting /content/RDD2022/RDD2022_all_countries/Czech.zip to /content/RDD2022/RDD2022_all_countries/Czech
total 2.0G
drwxr-xr-x 3 root root  4.0K Jun 18 09:43 China_Drone
-rw-r--r-- 1 root root  153M Jun 17 12:46 China_Drone.zip
drwxr-xr-x 3 root root  4.0K Jun 18 09:43 China_MotorBike
-rw-r--r-- 1 root root  184M Jun 17 12:46 China_MotorBike.zip
drwxr-xr-x 3 root root  4.0K Jun 18 09:43 Czech
-rw-r--r-- 1 root root  246M Jun 17 12:46 Czech.zip
drwxr-xr-x 3 root root  4.0K Jun 18 09:43 Japan
-rw-r--r-- 1 root roo

In [ ]:
# Dataset Root Directory
ROOTDIR = '/content/RDD2022/'

# Base Directory
CountryListDir = ["RDD2022_all_countries/Japan/Japan/train/annotations/xmls",
          "RDD2022_all_countries/China_MotorBike/China_MotorBike/train/annotations/xmls",
          "RDD2022_all_countries/Czech/Czech/train/annotations/xmls",
          "RDD2022_all_countries/United_States/United_States/train/annotations/xmls",]

for CountryDir in CountryListDir:

    CountryDir = ROOTDIR + CountryDir
    fileList = sorted(glob.glob(CountryDir + "/*.xml"))

    # Processing all the annotation
    for file in tqdm(fileList):
        convertPascal2YOLOv8(file)
        # break

100%|██████████| 10506/10506 [00:12<00:00, 829.91it/s]


In [ ]:
def CopyDatasetSplit(baseDir):

    random.seed(1337)

    # Output Directory
    baseOutputDir = '/content/drive/My Drive/AML/Dataset'
    countryName = Path(baseDir).parents[0]
    countryName = os.path.split(countryName)[1]

    baseImageDir = baseDir + "images/"
    baseAnnotDir = baseDir + "labels/"

    image_list_all = sorted(glob.glob(baseImageDir + "/*"))
    annot_list_all = sorted(glob.glob(baseAnnotDir + "/*"))

    # Drop any images that doesnt have annotation (background) by leaving it at some percentage of the dataset
    backgroundImages_Percentage = 0.1
    image_list = []
    annot_list = []

    dataset_length_all = len(image_list_all)
    max_background_image = int(dataset_length_all*backgroundImages_Percentage)
    _counter = 0

    for i in range(len(annot_list_all)):

        with open(annot_list_all[i]) as f:
            _annot = f.read()

            # Annotation not empty
            if _annot:
                image_list.append(image_list_all[i])
                annot_list.append(annot_list_all[i])
            elif _counter < max_background_image:
                image_list.append(image_list_all[i])
                annot_list.append(annot_list_all[i])
                _counter = _counter + 1

    # Dataset length
    dataset_length = len(image_list)
    # print(dataset_length, len(annot_list))

    split_ratio = 0.9
    middle_point = round(split_ratio * dataset_length)

    # Create random list number using seed
    numberList = list(range(0, dataset_length))
    random.shuffle(numberList)
    trainNumberList = numberList[:middle_point]
    validNumberList = numberList[middle_point:]
    print("Training/Validation Samples :", len(trainNumberList), len(validNumberList))

    # Training images and labels
    print("Copying training images and labels for", countryName)
    for i in tqdm(trainNumberList):

        # Images
        outputImagesDir = baseOutputDir + countryName + "/images/train/"
        if not os.path.exists(outputImagesDir):
            os.makedirs(outputImagesDir)

        shutil.copy2(image_list[i], outputImagesDir)

        # Annotations
        outputAnnotDir = baseOutputDir + countryName + "/labels/train/"
        if not os.path.exists(outputAnnotDir):
            os.makedirs(outputAnnotDir)

        shutil.copy2(annot_list[i], outputAnnotDir)
        print(outputImagesDir, outputAnnotDir)

    # Validation images and labels
    print("Copying validation images and labels for", countryName)
    for i in tqdm(validNumberList):

        # Images
        outputImagesDir = baseOutputDir + countryName + "/images/val/"
        if not os.path.exists(outputImagesDir):
            os.makedirs(outputImagesDir)

        shutil.copy2(image_list[i], outputImagesDir)

        # Annotations
        outputAnnotDir = baseOutputDir + countryName + "/labels/val/"
        if not os.path.exists(outputAnnotDir):
            os.makedirs(outputAnnotDir)

        shutil.copy2(annot_list[i], outputAnnotDir)
        print(outputImagesDir, outputAnnotDir)



In [ ]:
# Base Directory

ROOTDIR = '/content/RDD2022/'

# Use Japan,USA,Czech,ChinamotorBike
CountryListDir = ["RDD2022_all_countries/Japan/Japan/train/",
          "RDD2022_all_countries/China_MotorBike/China_MotorBike/train/",
          "RDD2022_all_countries/Czech/Czech/train/",
          "RDD2022_all_countries/United_States/United_States/train/"]

for CountryDir in CountryListDir:
    CountryDir = ROOTDIR + CountryDir
    CopyDatasetSplit(CountryDir)

Training/Validation Samples : 9455 1051
Copying training images and labels for Japan


  0%|          | 4/9455 [00:00<04:00, 39.23it/s]

/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/
/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/
/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/
/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/
/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/
/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/
/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/
/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/


  0%|          | 9/9455 [00:00<03:37, 43.43it/s]

/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/
/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/
/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/
/content/drive/My Drive/AML/DatasetJapan/images/train/ /content/drive/My Drive/AML/DatasetJapan/labels/train/


...

 65%|██████▌   | 1160/1779 [00:24<00:13, 44.78it/s]

/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/A

 66%|██████▌   | 1172/1779 [00:25<00:12, 46.72it/s]

/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/AML/DatasetChina_MotorBike/labels/train/
/content/drive/My Drive/AML/DatasetChina_MotorBike/images/train/ /content/drive/My Drive/A

KeyboardInterrupt: 